In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import data from C:\Data\hershey_promo_impact\shipment_hist_weekly_with_promo_features.csv
df = pd.read_csv("C:\\Data\\hershey_promo_impact\\shipment_hist_weekly_with_promo_features.csv")

Primary Key ['sales_org', 'dp_cust', 'from_loc', 'dmd_item_10', 'week_start_date']
qty: shipment quantity
po_qty_week_1: 1 weak ahead purchase order quantity
price_sch3: base price, time invariant
total_expense_trade'
forecast_cot_spend'

In [9]:
# Columns defining the new primary key
groupby_cols = ['sales_org', 'dp_cust', 'dmd_item_10', 'week_start_date']

# Numerical columns to sum
sum_columns = ['qty', 'po_qty_week_1', 'po_qty_week_2', 'po_qty_week_3', 'po_qty_week_4', 'po_qty_week_8']

# Non-numerical columns to pick the first non-missing value
first_non_missing_columns = [
    'descr', 'division', 'franchise', 'subbrand', 'brand', 'packtype',
    'ppg_family', 'ppg', 'season', 'sales_status', 'on_promotion',
    'promotion_type', 'total_expense_trade', 'forecast_cot_spend', 'price_sch3'
]

# Define aggregation rules
agg_dict = {col: 'sum' for col in sum_columns}
agg_dict.update({col: 'first' for col in first_non_missing_columns})

# Perform the aggregation
aggregated_df = df.groupby(groupby_cols).agg(agg_dict).reset_index()


In [10]:
# replace promotion_type with NA if on_promotion is 0
aggregated_df.loc[aggregated_df['on_promotion'] == False, 'promotion_type'] = np.nan

# create dummy variables for promotion_type
aggregated_df = pd.get_dummies(aggregated_df, columns=['promotion_type'], dummy_na=True)



In [11]:
# change data type for week_start_date to datetime
aggregated_df['week_start_date'] = pd.to_datetime(aggregated_df['week_start_date'])

# check data types for each column
print(aggregated_df.dtypes)

sales_org                                      object
dp_cust                                         int64
dmd_item_10                                     int64
week_start_date                        datetime64[ns]
qty                                           float64
po_qty_week_1                                 float64
po_qty_week_2                                 float64
po_qty_week_3                                 float64
po_qty_week_4                                 float64
po_qty_week_8                                 float64
descr                                          object
division                                       object
franchise                                      object
subbrand                                       object
brand                                          object
packtype                                      float64
ppg_family                                    float64
ppg                                            object
season                      

Panel Model

In [12]:
import pandas as pd
from linearmodels.panel import PanelOLS

# Step 1: Prepare the data
# Create a combined unit identifier
aggregated_df['unit_id'] = aggregated_df['sales_org'].astype(str) + '_' + \
                           aggregated_df['dp_cust'].astype(str) + '_' + \
                           aggregated_df['dmd_item_10'].astype(str)

# Set multi-index for panel data
aggregated_df = aggregated_df.set_index(['unit_id', 'week_start_date'])

# Ensure the dependent variable is numeric
aggregated_df['qty'] = pd.to_numeric(aggregated_df['qty'], errors='coerce')

# Step 2: Define independent variables (X)
x_columns = [
    'promotion_type_Corporate Promotions',
    'promotion_type_Correction',
    'promotion_type_EDLC',
    'promotion_type_EDLP',
    'promotion_type_Hi Lo',
    'promotion_type_Miscellaneous'
]
X = aggregated_df[x_columns]

# Step 3: Define dependent variable (y)
y = aggregated_df['qty']

# Step 4: Specify the panel model with fixed effects
model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# Step 5: Fit the model
results = model.fit(cov_type='clustered', cluster_entity=True)

# Step 6: Summarize the results
print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                    qty   R-squared:                        0.0003
Estimator:                   PanelOLS   R-squared (Between):             -0.0085
No. Observations:              363312   R-squared (Within):               0.0007
Date:                Wed, Jan 15 2025   R-squared (Overall):             -0.0028
Time:                        20:14:52   Log-likelihood                -3.303e+06
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      20.914
Entities:                        1682   P-value                           0.0000
Avg Obs:                       216.00   Distribution:                F(6,361409)
Min Obs:                       216.00                                           
Max Obs:                       216.00   F-statistic (robust):             5.1089
                            

panel for lead and lag of qty

In [13]:
import pandas as pd
from linearmodels.panel import PanelOLS


# Step 2: Create lag and lead variables
lags = [1, 2, 4, 8]
leads = [1, 2, 4, 8]

for lag in lags:
    aggregated_df[f'lag{lag}'] = aggregated_df.groupby(level='unit_id')['qty'].shift(lag)

for lead in leads:
    aggregated_df[f'lead{lead}'] = aggregated_df.groupby(level='unit_id')['qty'].shift(-lead)

# Step 3: Define independent variables (X)
x_columns = [
    'promotion_type_Corporate Promotions',
    'promotion_type_Correction',
    'promotion_type_EDLC',
    'promotion_type_EDLP',
    'promotion_type_Hi Lo',
    'promotion_type_Miscellaneous'
]
X = aggregated_df[x_columns]

# Step 4: Run panel models for each lag/lead variable
results_summary = []

for y_var in [f'lag{lag}' for lag in lags] + [f'lead{lead}' for lead in leads]:
    y = aggregated_df[y_var].dropna()
    X_subset = X.loc[y.index]  # Align X with the available y data

    # Fit the model
    model = PanelOLS(y, X_subset, entity_effects=True, time_effects=True)
    results = model.fit(cov_type='clustered', cluster_entity=True)

    # Store results
    summary_dict = {
        'Dependent Variable': y_var,
        'R-squared': results.rsquared,
        'Number of Observations': results.nobs
    }
    for var in x_columns:
        summary_dict[f'{var}_coef'] = results.params.get(var, None)
        summary_dict[f'{var}_pval'] = results.pvalues.get(var, None)
    results_summary.append(summary_dict)

# Step 5: Save results to Excel
results_df = pd.DataFrame(results_summary)
output_path = 'C:\\Users\\AdlerXie\\Documents\\GitHub\\Hershey_promo\\output\\panel_model_results.xlsx'
results_df.to_excel(output_path, index=False)

print(f"Results saved to {output_path}")


Results saved to C:\Users\AdlerXie\Documents\GitHub\Hershey_promo\output\panel_model_results.xlsx


In [38]:
#sort data by week_start_date, sales_org, dp_cust, dmd_item_10
aggregated_df = aggregated_df.sort_values(by=['week_start_date', 'sales_org', 'dp_cust', 'dmd_item_10'])


# save data to C:\Data\hershey_promo_impact\aggregated_shipment_hist_weekly_with_promo_features.csv
aggregated_df.to_csv("C:\\Data\\hershey_promo_impact\\aggregated_shipment_hist_weekly_with_promo_features.csv", index=False)